Run this for Kaggle setup with the Google API Key in the secrets

In [1]:
import os
from pathlib import Path

# Try to load API key from multiple sources
api_key_loaded = False

# Method 1: Check if already set in environment
if "GOOGLE_API_KEY" in os.environ:
    print("✅ GOOGLE_API_KEY found in environment variables")
    api_key_loaded = True

# Method 2: Try Kaggle secrets (if running in Kaggle)
if not api_key_loaded:
    try:
        from kaggle_secrets import UserSecretsClient
        GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        print("✅ Loaded GOOGLE_API_KEY from Kaggle secrets")
        api_key_loaded = True
    except ImportError:
        pass  # Not in Kaggle environment
    except Exception as e:
        print(f"⚠️  Could not load from Kaggle secrets: {e}")

# Method 3: Try .env file (for local development)
if not api_key_loaded:
    try:
        from dotenv import load_dotenv
        env_path = Path('.') / '.env'
        if env_path.exists():
            load_dotenv(env_path)
            if "GOOGLE_API_KEY" in os.environ:
                print("✅ Loaded GOOGLE_API_KEY from .env file")
                api_key_loaded = True
            else:
                print("⚠️  .env file exists but GOOGLE_API_KEY not found in it")
        else:
            print("⚠️  .env file not found")
    except ImportError:
        print("⚠️  python-dotenv not installed (optional for local development)")
    except Exception as e:
        print(f"⚠️  Error loading .env: {e}")

# Final check
if "GOOGLE_API_KEY" in os.environ:
    key_length = len(os.environ['GOOGLE_API_KEY'])
    print(f"✅ Setup complete - GOOGLE_API_KEY is set (length: {key_length})")
else:
    print("❌ GOOGLE_API_KEY is required but not found.")
    print("   Please set it using one of these methods:")
    print("   1. Kaggle: Add 'GOOGLE_API_KEY' to your Kaggle secrets")
    print("   2. Local: Create a .env file with: GOOGLE_API_KEY=your_key_here")
    print("   3. Environment: Set GOOGLE_API_KEY as an environment variable")

✅ GOOGLE_API_KEY found in environment variables
✅ Setup complete - GOOGLE_API_KEY is set (length: 39)


In [2]:
import json
import requests
import subprocess
import time
import uuid
import asyncio
import nest_asyncio

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

# Enable nested event loops for Jupyter notebooks
nest_asyncio.apply()

print("✅ ADK components imported successfully.")
print("✅ Async support enabled for Jupyter notebooks")

✅ ADK components imported successfully.
✅ Async support enabled for Jupyter notebooks


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# Focus Filter - Intelligent Notification Filtering Agent

This notebook implements a **multi-agent system** that intelligently filters and manages notifications by:
- Classifying notifications as urgent, irrelevant, or less urgent
- Taking appropriate actions (pass through, block, or store in memory)
- Learning from patterns to improve over time

## Contest Track: Concierge
## Key Concepts Demonstrated:
1. **Multi-agent system** (sequential agents: Classification → Action → Memory)
2. **Custom tools** for notification management
3. **Sessions & Memory** (long-term memory storage)
4. **Observability** (logging and tracing)
5. **Agent evaluation** (LLM-as-judge)

## Multi-Agent Architecture

This implementation uses a **sequential multi-agent system** with three specialized agents:

1. **Classification Agent**: Analyzes notifications and determines urgency category
2. **Action Agent**: Executes appropriate actions based on classification results
3. **Memory Agent**: Handles memory extraction, consolidation, and storage

The agents work sequentially: Classification → Action → Memory (when needed)


In [4]:
# ============================================================================
# Notification Data Structure and Memory Management
# ============================================================================

from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass

@dataclass
class Notification:
    """Represents a notification from an app or service"""
    id: str
    app: str
    title: str
    body: str
    timestamp: str
    category: Optional[str] = None
    
    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "app": self.app,
            "title": self.title,
            "body": self.body,
            "timestamp": self.timestamp,
            "category": self.category
        }
    
    def __str__(self) -> str:
        return f"[{self.app}] {self.title}: {self.body}"

# Simple in-memory storage for demonstration
class NotificationMemory:
    """Simple memory store for less urgent notifications"""
    def __init__(self):
        self.memories: List[Dict] = []
    
    def store(self, notification: Notification, extracted_fact: str):
        """Store a notification fact in memory"""
        memory = {
            "notification_id": notification.id,
            "app": notification.app,
            "extracted_fact": extracted_fact,
            "timestamp": notification.timestamp,
            "stored_at": datetime.now().isoformat()
        }
        self.memories.append(memory)
        print(f"💾 Stored memory: {extracted_fact}")
        return memory
    
    def get_all(self) -> List[Dict]:
        """Retrieve all stored memories"""
        return self.memories
    
    def search(self, query: str) -> List[Dict]:
        """Simple keyword search (would use vector DB in production)"""
        query_lower = query.lower()
        return [
            m for m in self.memories
            if query_lower in m["extracted_fact"].lower() or 
               query_lower in m["app"].lower()
        ]

# Initialize memory store
memory_store = NotificationMemory()
print("✅ Notification and Memory classes initialized")


✅ Notification and Memory classes initialized


In [5]:
# ============================================================================
# STEP 3 & 4: Enhanced Memory Management
# ============================================================================
# This cell enhances the NotificationMemory class with advanced features:
# - User preferences
# - Memory consolidation
# - Context compaction
# - Pattern learning

# Initialize enhanced attributes on the memory store
memory_store.user_preferences = {
    "always_urgent_apps": [],
    "always_block_apps": [],
    "preferred_categories": [],
    "blocked_keywords": []
}
memory_store.consolidation_threshold = 0.8

# Add enhanced methods to NotificationMemory
def _find_similar_memory(self, fact: str) -> Optional[Dict]:
    """Find similar memories to avoid duplicates"""
    fact_lower = fact.lower()
    fact_words = set(fact_lower.split())
    
    for memory in self.memories:
        existing_fact = memory["extracted_fact"].lower()
        existing_words = set(existing_fact.split())
        
        # Calculate simple similarity (Jaccard similarity)
        intersection = len(fact_words & existing_words)
        union = len(fact_words | existing_words)
        similarity = intersection / union if union > 0 else 0
        
        if similarity >= self.consolidation_threshold:
            return memory
    
    return None

def _merge_memories(self, memories: List[Dict]) -> Dict:
    """Merge multiple similar memories into one"""
    base = max(memories, key=lambda m: datetime.fromisoformat(m.get("last_updated", m["stored_at"])))
    total_count = sum(m.get("occurrence_count", 1) for m in memories)
    base["occurrence_count"] = total_count
    base["merged_from"] = len(memories)
    return base

def consolidate_memories(self):
    """Consolidate and merge similar memories"""
    consolidated = []
    processed = set()
    
    for i, memory in enumerate(self.memories):
        if i in processed:
            continue
        
        similar_group = [memory]
        for j, other_memory in enumerate(self.memories[i+1:], start=i+1):
            if j in processed:
                continue
            if self._find_similar_memory(other_memory["extracted_fact"]) == memory:
                similar_group.append(other_memory)
                processed.add(j)
        
        if len(similar_group) > 1:
            merged = self._merge_memories(similar_group)
            consolidated.append(merged)
            print(f"🔗 Consolidated {len(similar_group)} similar memories")
        else:
            consolidated.append(memory)
        
        processed.add(i)
    
    self.memories = consolidated
    return len(consolidated)

def get_user_preferences(self) -> Dict:
    """Get user preferences for notification filtering"""
    return self.user_preferences.copy()

def update_preference(self, preference_type: str, value: str, action: str = "add"):
    """Update user preferences (add or remove)"""
    if preference_type not in self.user_preferences:
        return {"status": "error", "message": f"Unknown preference type: {preference_type}"}
    
    if action == "add":
        if value not in self.user_preferences[preference_type]:
            self.user_preferences[preference_type].append(value)
            print(f"✅ Added preference: {preference_type} = {value}")
    elif action == "remove":
        if value in self.user_preferences[preference_type]:
            self.user_preferences[preference_type].remove(value)
            print(f"✅ Removed preference: {preference_type} = {value}")
    
    return {"status": "success", "preferences": self.user_preferences.copy()}

def learn_from_patterns(self, classification_history: List[Dict]):
    """Learn user preferences from classification patterns"""
    app_classifications = {}
    
    for entry in classification_history:
        app = entry.get("app", "")
        classification = entry.get("classification", "")
        
        if app not in app_classifications:
            app_classifications[app] = {"URGENT": 0, "IRRELEVANT": 0, "LESS_URGENT": 0}
        
        app_classifications[app][classification] = app_classifications[app].get(classification, 0) + 1
    
    # Auto-update preferences based on patterns
    for app, counts in app_classifications.items():
        total = sum(counts.values())
        if total >= 3:
            urgent_ratio = counts["URGENT"] / total
            irrelevant_ratio = counts["IRRELEVANT"] / total
            
            if urgent_ratio >= 0.8 and app not in self.user_preferences["always_urgent_apps"]:
                self.update_preference("always_urgent_apps", app, "add")
            elif irrelevant_ratio >= 0.8 and app not in self.user_preferences["always_block_apps"]:
                self.update_preference("always_block_apps", app, "add")

def compact_context(self, max_memories: int = 10) -> List[Dict]:
    """Compact context by returning most relevant memories"""
    sorted_memories = sorted(
        self.memories,
        key=lambda m: (
            datetime.fromisoformat(m.get("last_updated", m["stored_at"])),
            m.get("occurrence_count", 1)
        ),
        reverse=True
    )
    return sorted_memories[:max_memories]

# Add methods to NotificationMemory class
NotificationMemory._find_similar_memory = _find_similar_memory
NotificationMemory._merge_memories = _merge_memories
NotificationMemory.consolidate_memories = consolidate_memories
NotificationMemory.get_user_preferences = get_user_preferences
NotificationMemory.update_preference = update_preference
NotificationMemory.learn_from_patterns = learn_from_patterns
NotificationMemory.compact_context = compact_context

# Enhanced store method with deduplication
original_store = NotificationMemory.store
def enhanced_store(self, notification: Notification, extracted_fact: str):
    """Store with deduplication"""
    similar_memory = self._find_similar_memory(extracted_fact)
    
    if similar_memory:
        similar_memory["last_updated"] = datetime.now().isoformat()
        similar_memory["occurrence_count"] = similar_memory.get("occurrence_count", 1) + 1
        print(f"💾 Updated existing memory: {extracted_fact} (count: {similar_memory['occurrence_count']})")
        return similar_memory
    
    memory = original_store(self, notification, extracted_fact)
    memory["last_updated"] = datetime.now().isoformat()
    memory["occurrence_count"] = 1
    return memory

NotificationMemory.store = enhanced_store

print("✅ Enhanced memory management initialized")


✅ Enhanced memory management initialized


In [6]:
# ============================================================================
# STEP 3: Enhanced Tools - retrieve_user_preferences and Validation
# ============================================================================

def retrieve_user_preferences() -> dict:
    """
    Retrieve user preferences for notification filtering.
    This tool allows agents to access learned user preferences to make better decisions.
    
    Returns:
        Dictionary with user preferences including:
        - always_urgent_apps: Apps that should always be treated as urgent
        - always_block_apps: Apps that should always be blocked
        - preferred_categories: Categories the user cares about
        - blocked_keywords: Keywords that indicate blocking
        
        Success: {"status": "success", "preferences": {...}}
        Error: {"status": "error", "error_message": "..."}
    """
    try:
        preferences = memory_store.get_user_preferences()
        return {
            "status": "success",
            "preferences": preferences,
            "message": "User preferences retrieved successfully"
        }
    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Failed to retrieve preferences: {e}"
        }

# Enhanced tool validation wrapper
def validate_tool_input(func):
    """Decorator to validate tool inputs and handle errors"""
    def wrapper(*args, **kwargs):
        try:
            # Basic validation
            if not args and not kwargs:
                return {
                    "status": "error",
                    "error_message": "No arguments provided"
                }
            
            # Call the original function
            result = func(*args, **kwargs)
            
            # Validate result format
            if not isinstance(result, dict):
                return {
                    "status": "error",
                    "error_message": "Tool did not return a dictionary"
                }
            
            if "status" not in result:
                result["status"] = "success"
            
            return result
            
        except TypeError as e:
            return {
                "status": "error",
                "error_message": f"Invalid arguments: {e}"
            }
        except Exception as e:
            return {
                "status": "error",
                "error_message": f"Tool execution error: {e}"
            }
    
    return wrapper

# Apply validation to existing tools (optional - for demonstration)
# In production, you'd wrap the tools before passing to agents

print("✅ Enhanced tools ready:")
print("   • retrieve_user_preferences() - Access user preferences")
print("   • Tool validation framework available")


✅ Enhanced tools ready:
   • retrieve_user_preferences() - Access user preferences
   • Tool validation framework available


In [7]:
# Custom Tools for Notification Management

def display_urgent_notification(app: str, title: str, body: str) -> dict:
    """
    Display an urgent notification to the user immediately.
    This tool is called when a notification requires immediate attention.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        body: The notification body text
        
    Returns:
        Dictionary with status and result information.
        Success: {"status": "success", "message": "Notification displayed"}
        Error: {"status": "error", "error_message": "..."}
    """
    print(f"\n{'='*60}")
    print(f"🚨 URGENT NOTIFICATION")
    print(f"{'='*60}")
    print(f"App: {app}")
    print(f"Title: {title}")
    print(f"Body: {body}")
    print(f"{'='*60}\n")
    return {"status": "success", "message": f"Urgent notification from {app} displayed to user"}

def block_notification(app: str, title: str, reason: str) -> dict:
    """
    Block/suppress an irrelevant notification.
    This tool is called when a notification is determined to be noise.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        reason: The reason for blocking (e.g., "social media noise", "promotional content")
        
    Returns:
        Dictionary with status and result information.
        Success: {"status": "success", "message": "Notification blocked"}
        Error: {"status": "error", "error_message": "..."}
    """
    print(f"🚫 Blocked: [{app}] {title} - {reason}")
    return {"status": "success", "message": f"Notification from {app} blocked: {reason}"}

def save_notification_memory(app: str, title: str, body: str, extracted_fact: str) -> dict:
    """
    Save a less urgent notification as a memory for later review.
    This tool extracts key information and stores it for future reference.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        body: The notification body text
        extracted_fact: The key fact or information extracted from the notification
        
    Returns:
        Dictionary with status and result information.
        Success: {"status": "success", "message": "Memory stored"}
        Error: {"status": "error", "error_message": "..."}
    """
    # Create a temporary notification object for storage
    temp_notification = Notification(
        id=str(uuid.uuid4()),
        app=app,
        title=title,
        body=body,
        timestamp=datetime.now().isoformat()
    )
    memory_store.store(temp_notification, extracted_fact)
    return {"status": "success", "message": f"Memory stored: {extracted_fact}"}

print("✅ Custom tools defined")


✅ Custom tools defined


In [8]:
# ============================================================================
# MULTI-AGENT ARCHITECTURE: Sequential Agent System
# ============================================================================
# This demonstrates a multi-agent system with three specialized agents:
# 1. Classification Agent: Analyzes and classifies notifications
# 2. Action Agent: Executes actions based on classification
# 3. Memory Agent: Handles memory extraction and storage

# ----------------------------------------------------------------------------
# Agent 1: Classification Agent
# ----------------------------------------------------------------------------
# This agent's sole responsibility is to analyze notifications and determine
# their urgency category. It does NOT take actions, only classifies.

def classify_notification(app: str, title: str, body: str) -> dict:
    """
    Classify a notification into one of three categories: URGENT, IRRELEVANT, or LESS_URGENT.
    This tool is used by the Classification Agent to output its decision.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        body: The notification body text
        
    Returns:
        Dictionary with classification result.
    """
    # This is a tool that the classification agent will call to output its decision
    # The actual classification logic is in the agent's reasoning
    pass  # Placeholder - agent will call this with classification result

classification_agent = LlmAgent(
    name="classification_agent",
    model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
    instruction="""You are a Classification Agent in the Focus Filter system.

Your ONLY job is to analyze notifications and classify them into one of three categories:

1. **URGENT**: Requires immediate attention or action
   - Security alerts (bank, account access)
   - Critical deadlines or time-sensitive tasks
   - Emergency communications
   - Important personal messages requiring immediate response

2. **IRRELEVANT**: Noise that should be blocked
   - Social media likes, follows, generic updates
   - Marketing/promotional content
   - Low-value informational updates
   - Spam or unwanted notifications

3. **LESS_URGENT**: Important but not immediate - should be stored in memory
   - Project updates, deadline changes
   - Informational updates worth remembering
   - Non-critical but useful information
   - Things the user might want to reference later

When you receive a notification, analyze the app, title, and body text, then respond with:
- The classification (URGENT, IRRELEVANT, or LESS_URGENT)
- A brief reasoning for your decision
- If LESS_URGENT, also provide the key fact or information that should be extracted

Format your response as:
Classification: [URGENT/IRRELEVANT/LESS_URGENT]
Reasoning: [your reasoning]
Key Fact (if LESS_URGENT): [extracted fact]

Be conservative with URGENT - only use it for truly time-sensitive or critical items.""",
    tools=[],  # Classification agent doesn't take actions, only classifies
)

# ----------------------------------------------------------------------------
# Agent 2: Action Agent
# ----------------------------------------------------------------------------
# This agent receives classification results and executes the appropriate action

action_agent = LlmAgent(
    name="action_agent",
    model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
    instruction="""You are an Action Agent in the Focus Filter system.

Your job is to execute actions based on classification results from the Classification Agent.

You will receive:
- The notification details (app, title, body)
- The classification result (URGENT, IRRELEVANT, or LESS_URGENT)
- Any additional context (like extracted facts for LESS_URGENT items)

Based on the classification, you must:
1. For URGENT: Call `display_urgent_notification(app, title, body)`
2. For IRRELEVANT: Call `block_notification(app, title, reason)` with a clear reason
3. For LESS_URGENT: Call `save_notification_memory(app, title, body, extracted_fact)` with the key fact

Execute the appropriate action immediately based on the classification provided.""",
    tools=[
        display_urgent_notification,
        block_notification,
        save_notification_memory,
    ],
)

# ----------------------------------------------------------------------------
# Agent 3: Memory Agent (for advanced memory operations)
# ----------------------------------------------------------------------------
# This agent handles memory extraction, consolidation, and retrieval

def extract_memory_fact(app: str, title: str, body: str) -> dict:
    """
    Extract the key fact or information from a notification for memory storage.
    This tool is used by the Memory Agent to extract structured information.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        body: The notification body text
        
    Returns:
        Dictionary with extracted fact.
    """
    # This tool allows the memory agent to output extracted facts
    pass  # Placeholder - agent will call this with extracted fact

memory_agent = LlmAgent(
    name="memory_agent",
    model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
    instruction="""You are a Memory Agent in the Focus Filter system.

Your job is to extract, consolidate, and manage memories from notifications.

When you receive a LESS_URGENT notification:
1. Extract the most important fact or piece of information
2. Format it as a concise, searchable memory
3. Ensure it's useful for future reference

For example:
- "Project deadline moved to Tuesday" (not "Your project deadline has moved to Tuesday")
- "New team member joined: Alice" (not the full notification text)

Focus on extracting actionable, referenceable facts that the user might want to recall later.""",
    tools=[],  # Memory agent primarily extracts facts (can be extended with retrieval tools)
)

print("✅ Multi-agent system created:")
print("  • Classification Agent - Analyzes and classifies notifications")
print("  • Action Agent - Executes actions based on classification")
print("  • Memory Agent - Handles memory extraction and consolidation")

# Update agents to include retrieve_user_preferences tool
# Add the tool to classification and action agents
if retrieve_user_preferences not in classification_agent.tools:
    classification_agent.tools.append(retrieve_user_preferences)

if retrieve_user_preferences not in action_agent.tools:
    action_agent.tools.append(retrieve_user_preferences)

print("✅ Agents updated with enhanced tools")
print("   • Classification Agent can now check user preferences")
print("   • Action Agent can now check user preferences")


✅ Multi-agent system created:
  • Classification Agent - Analyzes and classifies notifications
  • Action Agent - Executes actions based on classification
  • Memory Agent - Handles memory extraction and consolidation
✅ Agents updated with enhanced tools
   • Classification Agent can now check user preferences
   • Action Agent can now check user preferences


In [9]:
# ============================================================================
# OBSERVABILITY: Logging, Tracing, and Metrics
# ============================================================================
# This module provides comprehensive observability for the multi-agent system

import logging
from collections import defaultdict
from datetime import datetime
from typing import Dict, List, Optional
import json

# Configure structured logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("FocusFilter")

class ObservabilityManager:
    """Manages logging, tracing, and metrics for the multi-agent system"""
    
    def __init__(self):
        self.traces: List[Dict] = []
        self.metrics = {
            "total_notifications": 0,
            "classifications": defaultdict(int),
            "actions": defaultdict(int),
            "agent_timings": defaultdict(list),
            "errors": []
        }
        self.current_trace: Optional[Dict] = None
    
    def start_trace(self, notification_id: str, notification_text: str):
        """Start a new trace for a notification processing session"""
        self.current_trace = {
            "trace_id": f"trace_{uuid.uuid4().hex[:8]}",
            "notification_id": notification_id,
            "notification_text": notification_text,
            "start_time": datetime.now().isoformat(),
            "agents": [],
            "classification": None,
            "action": None,
            "memory_operation": None,
            "end_time": None,
            "duration_ms": None,
            "errors": []
        }
        logger.info(f"🔍 Trace started: {self.current_trace['trace_id']}")
    
    def log_agent_step(self, agent_name: str, step: str, input_data: Dict, output_data: Dict, duration_ms: float):
        """Log an agent step in the current trace"""
        if self.current_trace is None:
            return
        
        agent_step = {
            "agent": agent_name,
            "step": step,
            "timestamp": datetime.now().isoformat(),
            "input": input_data,
            "output": output_data,
            "duration_ms": duration_ms
        }
        self.current_trace["agents"].append(agent_step)
        self.metrics["agent_timings"][f"{agent_name}_{step}"].append(duration_ms)
        
        logger.info(f"📊 {agent_name} - {step} completed in {duration_ms:.2f}ms")
    
    def log_classification(self, classification: str, reasoning: str, confidence: Optional[float] = None):
        """Log classification result"""
        if self.current_trace is None:
            return
        
        self.current_trace["classification"] = {
            "result": classification,
            "reasoning": reasoning,
            "confidence": confidence,
            "timestamp": datetime.now().isoformat()
        }
        self.metrics["classifications"][classification] += 1
        
        logger.info(f"🏷️  Classification: {classification} - {reasoning[:100]}")
    
    def log_action(self, action_type: str, action_details: Dict):
        """Log action execution"""
        if self.current_trace is None:
            return
        
        self.current_trace["action"] = {
            "type": action_type,
            "details": action_details,
            "timestamp": datetime.now().isoformat()
        }
        self.metrics["actions"][action_type] += 1
        
        logger.info(f"⚡ Action: {action_type} - {json.dumps(action_details, default=str)}")
    
    def log_memory_operation(self, operation: str, details: Dict):
        """Log memory operation"""
        if self.current_trace is None:
            return
        
        self.current_trace["memory_operation"] = {
            "operation": operation,
            "details": details,
            "timestamp": datetime.now().isoformat()
        }
        
        logger.info(f"💾 Memory: {operation} - {json.dumps(details, default=str)}")
    
    def log_error(self, error_type: str, error_message: str, context: Optional[Dict] = None):
        """Log an error"""
        error_entry = {
            "type": error_type,
            "message": error_message,
            "context": context or {},
            "timestamp": datetime.now().isoformat()
        }
        self.metrics["errors"].append(error_entry)
        
        if self.current_trace:
            self.current_trace["errors"].append(error_entry)
        
        logger.error(f"❌ Error [{error_type}]: {error_message}")
    
    def end_trace(self):
        """End the current trace and store it"""
        if self.current_trace is None:
            return
        
        end_time = datetime.now()
        start_time = datetime.fromisoformat(self.current_trace["start_time"])
        duration_ms = (end_time - start_time).total_seconds() * 1000
        
        self.current_trace["end_time"] = end_time.isoformat()
        self.current_trace["duration_ms"] = duration_ms
        
        self.traces.append(self.current_trace.copy())
        self.metrics["total_notifications"] += 1
        
        logger.info(f"✅ Trace completed: {self.current_trace['trace_id']} in {duration_ms:.2f}ms")
        self.current_trace = None
    
    def get_metrics_summary(self) -> Dict:
        """Get a summary of all metrics"""
        avg_timings = {}
        for key, timings in self.metrics["agent_timings"].items():
            if timings:
                avg_timings[key] = {
                    "avg_ms": sum(timings) / len(timings),
                    "min_ms": min(timings),
                    "max_ms": max(timings),
                    "count": len(timings)
                }
        
        return {
            "total_notifications": self.metrics["total_notifications"],
            "classifications": dict(self.metrics["classifications"]),
            "actions": dict(self.metrics["actions"]),
            "average_timings": avg_timings,
            "error_count": len(self.metrics["errors"]),
            "total_traces": len(self.traces)
        }
    
    def get_recent_traces(self, limit: int = 10) -> List[Dict]:
        """Get the most recent traces"""
        return self.traces[-limit:] if self.traces else []
    
    def print_metrics_summary(self):
        """Print a formatted metrics summary"""
        summary = self.get_metrics_summary()
        
        print("\n" + "="*70)
        print("📊 OBSERVABILITY METRICS SUMMARY")
        print("="*70)
        print(f"\n📈 Total Notifications Processed: {summary['total_notifications']}")
        print(f"📝 Total Traces Captured: {summary['total_traces']}")
        
        print("\n🏷️  Classification Distribution:")
        for classification, count in summary['classifications'].items():
            percentage = (count / summary['total_notifications'] * 100) if summary['total_notifications'] > 0 else 0
            print(f"   {classification}: {count} ({percentage:.1f}%)")
        
        print("\n⚡ Action Distribution:")
        for action, count in summary['actions'].items():
            percentage = (count / summary['total_notifications'] * 100) if summary['total_notifications'] > 0 else 0
            print(f"   {action}: {count} ({percentage:.1f}%)")
        
        if summary['average_timings']:
            print("\n⏱️  Performance Metrics:")
            for key, timing in summary['average_timings'].items():
                print(f"   {key}:")
                print(f"      Average: {timing['avg_ms']:.2f}ms")
                print(f"      Min: {timing['min_ms']:.2f}ms")
                print(f"      Max: {timing['max_ms']:.2f}ms")
                print(f"      Count: {timing['count']}")
        
        if summary['error_count'] > 0:
            print(f"\n❌ Errors: {summary['error_count']}")
            for error in self.metrics['errors'][-5:]:  # Show last 5 errors
                print(f"   [{error['type']}] {error['message']}")
        
        print("="*70 + "\n")

# Initialize observability manager
obs_manager = ObservabilityManager()

print("✅ Observability system initialized")
print("   • Structured logging enabled")
print("   • Trace capture ready")
print("   • Metrics collection active")


✅ Observability system initialized
   • Structured logging enabled
   • Trace capture ready
   • Metrics collection active


In [10]:
# ============================================================================
# ORCHESTRATION LAYER: Sequential Agent Coordination
# ============================================================================
# This layer coordinates the sequential flow: Classification → Action → Memory

# Create runners for each agent
classification_runner = Runner(
    app_name="FocusFilter_Classification",
    agent=classification_agent,
    session_service=InMemorySessionService(),
)

action_runner = Runner(
    app_name="FocusFilter_Action",
    agent=action_agent,
    session_service=InMemorySessionService(),
)

memory_runner = Runner(
    app_name="FocusFilter_Memory",
    agent=memory_agent,
    session_service=InMemorySessionService(),
)

# Main session service for the orchestration
session_service = InMemorySessionService()

def parse_classification_result(text: str) -> dict:
    """Parse the classification agent's response to extract structured data"""
    result = {
        "classification": None,
        "reasoning": None,
        "key_fact": None
    }
    
    text_lower = text.lower()
    
    # Try to find classification - look for URGENT, IRRELEVANT, or LESS_URGENT
    for classification in ["URGENT", "IRRELEVANT", "LESS_URGENT", "LESS URGENT"]:
        if classification.lower() in text_lower:
            # Check if it's in a structured format like "Classification: URGENT"
            if "classification:" in text_lower:
                idx = text_lower.find("classification:")
                after_colon = text[text_lower.find("classification:") + len("classification:"):].strip()
                # Extract the first word or phrase after the colon
                words = after_colon.split()
                if words:
                    result["classification"] = words[0].upper().rstrip(".,;")
                    break
            else:
                # Look for the classification word in context
                result["classification"] = classification.upper()
                break
    
    # Extract reasoning
    if "reasoning:" in text_lower:
        idx = text_lower.find("reasoning:")
        reasoning_text = text[idx + len("reasoning:"):].strip()
        # Take up to the next section or end of text
        next_section = min(
            reasoning_text.find("\n\n"),
            reasoning_text.find("Key Fact"),
            reasoning_text.find("key fact"),
            len(reasoning_text)
        )
        if next_section > 0:
            result["reasoning"] = reasoning_text[:next_section].strip()
        else:
            result["reasoning"] = reasoning_text.strip()
    
    # Extract key fact (for LESS_URGENT items)
    if "key fact" in text_lower:
        idx = text_lower.find("key fact")
        fact_text = text[idx + len("key fact"):].strip()
        # Remove colon if present
        if fact_text.startswith(":"):
            fact_text = fact_text[1:].strip()
        # Take up to next line break or end
        next_line = fact_text.find("\n")
        if next_line > 0:
            result["key_fact"] = fact_text[:next_line].strip()
        else:
            result["key_fact"] = fact_text.strip()
    
    # Fallback: if classification not found, try to infer from text
    if result["classification"] is None:
        if any(word in text_lower for word in ["urgent", "immediate", "critical", "security alert"]):
            result["classification"] = "URGENT"
        elif any(word in text_lower for word in ["irrelevant", "noise", "block", "spam"]):
            result["classification"] = "IRRELEVANT"
        elif any(word in text_lower for word in ["less urgent", "store", "memory", "later"]):
            result["classification"] = "LESS_URGENT"
    
    return result

async def process_notification_multi_agent(
    notification_text: str,
    user_id: str = "default",
    session_id: str = None
):
    """
    Orchestrate the sequential multi-agent processing of a notification.
    
    Flow:
    1. Classification Agent analyzes and classifies
    2. Action Agent executes appropriate action
    3. Memory Agent (if needed) handles memory extraction
    
    Args:
        notification_text: The notification message to process
        user_id: User identifier
        session_id: Optional session identifier (auto-generated if None)
    """
    import time
    
    if session_id is None:
        session_id = f"session_{uuid.uuid4().hex[:8]}"
    
    # Start observability trace
    notification_id = f"notif_{uuid.uuid4().hex[:8]}"
    obs_manager.start_trace(notification_id, notification_text)
    
    print(f"\n{'='*70}")
    print(f"🔍 Processing notification with multi-agent system")
    print(f"{'='*70}\n")
    
    # Step 1: Classification Agent
    print("📊 Step 1: Classification Agent analyzing...")
    classification_start = time.time()
    classification_result_text = ""
    
    try:
        session = await classification_runner.session_service.create_session(
            app_name=classification_runner.app_name,
            user_id=user_id,
            session_id=f"{session_id}_classify"
        )
    except Exception as e:
        obs_manager.log_error("session_creation", f"Failed to create classification session: {e}")
        session = await classification_runner.session_service.get_session(
            app_name=classification_runner.app_name,
            user_id=user_id,
            session_id=f"{session_id}_classify"
        )
    
    message = types.Content(
        role="user",
        parts=[types.Part(text=notification_text)]
    )
    
    try:
        async for event in classification_runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=message
        ):
            if event.content and event.content.parts:
                if event.content.parts[0].text:
                    classification_result_text += event.content.parts[0].text
    except Exception as e:
        obs_manager.log_error("classification_error", f"Classification agent error: {e}", {"session_id": session_id})
        raise
    
    classification_duration = (time.time() - classification_start) * 1000
    print(f"✅ Classification complete")
    print(f"   Result: {classification_result_text[:200]}...\n")
    
    # Parse classification result
    classification_data = parse_classification_result(classification_result_text)
    
    # Log classification step
    obs_manager.log_agent_step(
        agent_name="Classification Agent",
        step="classify",
        input_data={"notification_text": notification_text[:200]},
        output_data=classification_data,
        duration_ms=classification_duration
    )
    
    # Log classification result
    if classification_data['classification']:
        obs_manager.log_classification(
            classification=classification_data['classification'],
            reasoning=classification_data.get('reasoning', 'No reasoning provided')
        )
    
    # Extract notification details from input
    # Format: "I received a notification: App: X, Title: Y, Body: Z"
    app = "Unknown"
    title = "Unknown"
    body = "Unknown"
    
    if "App:" in notification_text:
        parts = notification_text.split("App:")[-1]
        if "Title:" in parts:
            app = parts.split("Title:")[0].strip().rstrip(",")
            title_part = parts.split("Title:")[-1]
            if "Body:" in title_part:
                title = title_part.split("Body:")[0].strip().rstrip(",")
                body = title_part.split("Body:")[-1].strip()
    
    # Step 2: Action Agent
    print("⚡ Step 2: Action Agent executing...")
    action_start = time.time()
    
    # Prepare action message with classification result
    action_message_text = f"""Notification Details:
- App: {app}
- Title: {title}
- Body: {body}

Classification Result:
- Classification: {classification_data['classification']}
- Reasoning: {classification_data['reasoning']}
{f"- Key Fact: {classification_data['key_fact']}" if classification_data['key_fact'] else ""}

Please execute the appropriate action based on the classification."""
    
    try:
        action_session = await action_runner.session_service.create_session(
            app_name=action_runner.app_name,
            user_id=user_id,
            session_id=f"{session_id}_action"
        )
    except Exception as e:
        obs_manager.log_error("session_creation", f"Failed to create action session: {e}")
        action_session = await action_runner.session_service.get_session(
            app_name=action_runner.app_name,
            user_id=user_id,
            session_id=f"{session_id}_action"
        )
    
    action_message = types.Content(
        role="user",
        parts=[types.Part(text=action_message_text)]
    )
    
    action_output = ""
    try:
        async for event in action_runner.run_async(
            user_id=user_id,
            session_id=action_session.id,
            new_message=action_message
        ):
            if event.content and event.content.parts:
                if event.content.parts[0].text:
                    action_output += event.content.parts[0].text
                    print(event.content.parts[0].text)
    except Exception as e:
        obs_manager.log_error("action_error", f"Action agent error: {e}", {"session_id": session_id})
        raise
    
    action_duration = (time.time() - action_start) * 1000
    
    # Determine action type from output
    action_type = "unknown"
    if "display_urgent_notification" in action_output.lower() or "urgent notification" in action_output.lower():
        action_type = "display_urgent"
    elif "block" in action_output.lower() or "blocked" in action_output.lower():
        action_type = "block"
    elif "save" in action_output.lower() or "memory" in action_output.lower() or "stored" in action_output.lower():
        action_type = "save_memory"
    
    # Log action step
    obs_manager.log_agent_step(
        agent_name="Action Agent",
        step="execute",
        input_data={
            "classification": classification_data['classification'],
            "app": app,
            "title": title
        },
        output_data={"action_type": action_type, "output": action_output[:200]},
        duration_ms=action_duration
    )
    
    # Log action
    obs_manager.log_action(
        action_type=action_type,
        action_details={
            "app": app,
            "title": title,
            "classification": classification_data['classification']
        }
    )
    
    print(f"✅ Action execution complete\n")
    
    # Step 3: Memory Agent (only for LESS_URGENT items that need extraction refinement)
    if classification_data['classification'] == 'LESS_URGENT':
        print("💾 Step 3: Memory Agent extracting fact...")
        memory_start = time.time()
        
        # Log memory operation
        if classification_data.get('key_fact'):
            obs_manager.log_memory_operation(
                operation="store",
                details={
                    "app": app,
                    "extracted_fact": classification_data['key_fact'],
                    "title": title
                }
            )
        
        memory_duration = (time.time() - memory_start) * 1000
        obs_manager.log_agent_step(
            agent_name="Memory Agent",
            step="extract",
            input_data={"notification": f"{app}: {title}"},
            output_data={"extracted_fact": classification_data.get('key_fact', '')},
            duration_ms=memory_duration
        )
        
        # The action agent already stored the memory, but we can use memory agent
        # for additional processing if needed (consolidation, deduplication, etc.)
        # For now, we'll skip this step as the action agent handles storage
        print("✅ Memory extraction handled by Action Agent\n")
    
    # End trace
    obs_manager.end_trace()
    
    print(f"{'='*70}")
    print(f"✅ Multi-agent processing complete")
    print(f"{'='*70}\n")

# Legacy helper function for backward compatibility (uses single agent approach)
async def run_notification_test(runner_instance, session_service, user_id, session_id, message_text):
    """Helper function to test notification processing (legacy single-agent)"""
    # Create or get session
    try:
        session = await session_service.create_session(
            app_name=runner_instance.app_name, user_id=user_id, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=runner_instance.app_name, user_id=user_id, session_id=session_id
        )
    
    # Convert message to Content format
    message = types.Content(
        role="user",
        parts=[types.Part(text=message_text)]
    )
    
    # Process the notification
    async for event in runner_instance.run_async(
        user_id=user_id, session_id=session.id, new_message=message
    ):
        if event.content and event.content.parts:
            if event.content.parts[0].text:
                print(event.content.parts[0].text)

print("✅ Multi-agent orchestration layer initialized")
print("✅ Sequential agent coordination ready")
print("✅ Helper functions for async operations ready")


✅ Multi-agent orchestration layer initialized
✅ Sequential agent coordination ready
✅ Helper functions for async operations ready


## Testing the Agent

Let's test the agent with sample notifications:


In [11]:
# Test notification 1: Urgent security alert
USER_ID = "test"

await process_notification_multi_agent(
    "I received a notification: App: Banking App, Title: Security Alert, Body: Your bank flagged suspicious activity on your account. Please verify immediately.",
    user_id=USER_ID,
    session_id="test_session_1"
)

print("\n" + "="*70)
print("Test 1 Complete")
print("="*70)


2025-12-02 02:43:22,752 - FocusFilter - INFO - 🔍 Trace started: trace_6342aa54



🔍 Processing notification with multi-agent system

📊 Step 1: Classification Agent analyzing...


2025-12-02 02:43:23,234 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:43:23,980 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:23,991 - FocusFilter - INFO - 📊 Classification Agent - classify completed in 1236.03ms
2025-12-02 02:43:23,996 - FocusFilter - INFO - 🏷️  Classification: URGENT - Security alert from a banking app requires immediate attention to prevent potential fraud.


✅ Classification complete
   Result: Classification: URGENT
Reasoning: Security alert from a banking app requires immediate attention to prevent potential fraud.
...

⚡ Step 2: Action Agent executing...


2025-12-02 02:43:24,330 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:43:25,161 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:25,163 - google_genai.types - WARNING - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-12-02 02:43:25,174 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False



🚨 URGENT NOTIFICATION
App: Banking App
Title: Security Alert
Body: Your bank flagged suspicious activity on your account. Please verify immediately.



2025-12-02 02:43:25,807 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:25,810 - FocusFilter - INFO - 📊 Action Agent - execute completed in 1808.52ms
2025-12-02 02:43:25,811 - FocusFilter - INFO - ⚡ Action: display_urgent - {"app": "Banking App", "title": "Security Alert", "classification": "URGENT"}
2025-12-02 02:43:25,813 - FocusFilter - INFO - ✅ Trace completed: trace_6342aa54 in 3061.68ms


OK. I have displayed the urgent notification to the user.

✅ Action execution complete

✅ Multi-agent processing complete


Test 1 Complete


In [12]:
# Test notification 2: Irrelevant social media
await process_notification_multi_agent(
    "I received a notification: App: Social Media, Title: New Like, Body: 3 new people liked your photo.",
    user_id=USER_ID,
    session_id="test_session_2"
)

print("\n" + "="*70)
print("Test 2 Complete")
print("="*70)


2025-12-02 02:43:25,833 - FocusFilter - INFO - 🔍 Trace started: trace_86cb1c48
2025-12-02 02:43:25,839 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False



🔍 Processing notification with multi-agent system

📊 Step 1: Classification Agent analyzing...


2025-12-02 02:43:26,491 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:26,495 - FocusFilter - INFO - 📊 Classification Agent - classify completed in 658.09ms
2025-12-02 02:43:26,496 - FocusFilter - INFO - 🏷️  Classification: IRRELEVANT - Social media likes are generally not important or time-sensitive.
2025-12-02 02:43:26,501 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False


✅ Classification complete
   Result: Classification: IRRELEVANT
Reasoning: Social media likes are generally not important or time-sensitive.
...

⚡ Step 2: Action Agent executing...


2025-12-02 02:43:27,209 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:27,211 - google_genai.types - WARNING - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-12-02 02:43:27,224 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False


🚫 Blocked: [Social Media] New Like - social media noise


2025-12-02 02:43:27,866 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:27,871 - FocusFilter - INFO - 📊 Action Agent - execute completed in 1372.97ms
2025-12-02 02:43:27,872 - FocusFilter - INFO - ⚡ Action: block - {"app": "Social Media", "title": "New Like", "classification": "IRRELEVANT"}
2025-12-02 02:43:27,873 - FocusFilter - INFO - ✅ Trace completed: trace_86cb1c48 in 2040.36ms


OK. I have blocked the notification from Social Media with the title "New Like" because it is social media noise.

✅ Action execution complete

✅ Multi-agent processing complete


Test 2 Complete


In [13]:
# Display observability metrics
obs_manager.print_metrics_summary()



📊 OBSERVABILITY METRICS SUMMARY

📈 Total Notifications Processed: 2
📝 Total Traces Captured: 2

🏷️  Classification Distribution:
   URGENT: 1 (50.0%)
   IRRELEVANT: 1 (50.0%)

⚡ Action Distribution:
   display_urgent: 1 (50.0%)
   block: 1 (50.0%)

⏱️  Performance Metrics:
   Classification Agent_classify:
      Average: 947.06ms
      Min: 658.09ms
      Max: 1236.03ms
      Count: 2
   Action Agent_execute:
      Average: 1590.75ms
      Min: 1372.97ms
      Max: 1808.52ms
      Count: 2



In [14]:
# Display a detailed trace example
print("\n" + "="*70)
print("🔍 DETAILED TRACE EXAMPLE")
print("="*70)

recent_traces = obs_manager.get_recent_traces(limit=1)
if recent_traces:
    trace = recent_traces[0]
    print(f"\nTrace ID: {trace['trace_id']}")
    print(f"Notification ID: {trace['notification_id']}")
    print(f"Duration: {trace['duration_ms']:.2f}ms")
    print(f"\nNotification Text: {trace['notification_text'][:100]}...")
    
    if trace['classification']:
        print(f"\n📊 Classification:")
        print(f"   Result: {trace['classification']['result']}")
        print(f"   Reasoning: {trace['classification']['reasoning'][:150]}...")
    
    if trace['action']:
        print(f"\n⚡ Action:")
        print(f"   Type: {trace['action']['type']}")
        print(f"   Details: {trace['action']['details']}")
    
    if trace['memory_operation']:
        print(f"\n💾 Memory Operation:")
        print(f"   Operation: {trace['memory_operation']['operation']}")
        print(f"   Details: {trace['memory_operation']['details']}")
    
    if trace['agents']:
        print(f"\n🤖 Agent Steps:")
        for agent_step in trace['agents']:
            print(f"   {agent_step['agent']} - {agent_step['step']}: {agent_step['duration_ms']:.2f}ms")
    
    if trace['errors']:
        print(f"\n❌ Errors: {len(trace['errors'])}")
        for error in trace['errors']:
            print(f"   [{error['type']}] {error['message']}")
else:
    print("No traces available yet. Run some notifications first!")

print("="*70 + "\n")



🔍 DETAILED TRACE EXAMPLE

Trace ID: trace_86cb1c48
Notification ID: notif_aabb98dc
Duration: 2040.36ms

Notification Text: I received a notification: App: Social Media, Title: New Like, Body: 3 new people liked your photo....

📊 Classification:
   Result: IRRELEVANT
   Reasoning: Social media likes are generally not important or time-sensitive....

⚡ Action:
   Type: block
   Details: {'app': 'Social Media', 'title': 'New Like', 'classification': 'IRRELEVANT'}

🤖 Agent Steps:
   Classification Agent - classify: 658.09ms
   Action Agent - execute: 1372.97ms



In [15]:
# Test notification 3: Less urgent project update
await process_notification_multi_agent(
    "I received a notification: App: Project Manager, Title: Deadline Update, Body: Your project deadline has moved to Tuesday.",
    user_id=USER_ID,
    session_id="test_session_3"
)

print("\n" + "="*70)
print("Test 3 Complete")
print("="*70)


2025-12-02 02:43:27,938 - FocusFilter - INFO - 🔍 Trace started: trace_c13ff761
2025-12-02 02:43:27,944 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False



🔍 Processing notification with multi-agent system

📊 Step 1: Classification Agent analyzing...


2025-12-02 02:43:28,607 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:28,613 - FocusFilter - INFO - 📊 Classification Agent - classify completed in 670.91ms
2025-12-02 02:43:28,617 - FocusFilter - INFO - 🏷️  Classification: LESS_URGENT - The notification is providing an update on a project deadline, which is important to know but doesn'
2025-12-02 02:43:28,639 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False


✅ Classification complete
   Result: Classification: LESS_URGENT
Reasoning: The notification is providing an update on a project deadline, which is important to know but doesn't require immediate action.
Key Fact (if LESS_URGENT): Projec...

⚡ Step 2: Action Agent executing...


2025-12-02 02:43:29,563 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:29,565 - google_genai.types - WARNING - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-12-02 02:43:29,579 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False


💾 Stored memory: Project deadline moved to Tuesday.


2025-12-02 02:43:30,180 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:30,187 - FocusFilter - INFO - 📊 Action Agent - execute completed in 1559.93ms
2025-12-02 02:43:30,190 - FocusFilter - INFO - ⚡ Action: save_memory - {"app": "Project Manager", "title": "Deadline Update", "classification": "LESS_URGENT"}
2025-12-02 02:43:30,194 - FocusFilter - INFO - 💾 Memory: store - {"app": "Project Manager", "extracted_fact": "(if LESS_URGENT): Project deadline moved to Tuesday.", "title": "Deadline Update"}
2025-12-02 02:43:30,197 - FocusFilter - INFO - 📊 Memory Agent - extract completed in 3.48ms
2025-12-02 02:43:30,200 - FocusFilter - INFO - ✅ Trace completed: trace_c13ff761 in 2262.49ms


OK. I have saved the notification to memory with the key fact "Project deadline moved to Tuesday."

✅ Action execution complete

💾 Step 3: Memory Agent extracting fact...
✅ Memory extraction handled by Action Agent

✅ Multi-agent processing complete


Test 3 Complete


In [16]:
# Display stored memories
print("\n" + "="*70)
print("💾 Stored Memories:")
print("="*70)

memories = memory_store.get_all()
if memories:
    for i, memory in enumerate(memories, 1):
        print(f"\n{i}. {memory['extracted_fact']}")
        print(f"   From: {memory['app']} (stored at {memory['stored_at']})")
else:
    print("No memories stored yet.")



💾 Stored Memories:

1. Project deadline moved to Tuesday.
   From: Project Manager (stored at 2025-12-02T02:43:29.575162)


In [17]:
# ============================================================================
# DEMONSTRATION: Enhanced Memory and Tools Features
# ============================================================================

print("\n" + "="*70)
print("🔧 Testing Enhanced Features")
print("="*70)

# Test 1: Retrieve user preferences
print("\n1. Testing retrieve_user_preferences() tool:")
prefs = retrieve_user_preferences()
if prefs.get('status') == 'success' and 'preferences' in prefs:
    print(f"   Current preferences: {prefs['preferences']}")
else:
    error_msg = prefs.get('error_message', 'Unknown error')
    print(f"   Error: {error_msg}")
    # Fallback: try direct access if method exists
    if hasattr(memory_store, 'get_user_preferences'):
        try:
            direct_prefs = memory_store.get_user_preferences()
            print(f"   Direct access preferences: {direct_prefs}")
        except Exception as e:
            print(f"   Direct access failed: {e}")
    else:
        print("   Note: Enhanced methods not yet available (run enhancement cell first)")

# Test 2: Update preferences
print("\n2. Testing preference learning:")
if hasattr(memory_store, 'update_preference'):
    memory_store.update_preference("always_urgent_apps", "Banking App", "add")
    memory_store.update_preference("always_block_apps", "Social Media", "add")
    if hasattr(memory_store, 'get_user_preferences'):
        print(f"   Updated preferences: {memory_store.get_user_preferences()}")
    else:
        print("   Preferences updated (get_user_preferences not available)")
else:
    print("   Note: update_preference method not available (run enhancement cell first)")

# Test 3: Memory consolidation
print("\n3. Testing memory consolidation:")
if hasattr(memory_store, 'consolidate_memories'):
    # Add a similar memory to test consolidation
    if memory_store.memories:
        test_notification = Notification(
            id=str(uuid.uuid4()),
            app="Project Manager",
            title="Deadline Update",
            body="Your project deadline has moved to Tuesday.",
            timestamp=datetime.now().isoformat()
        )
        memory_store.store(test_notification, "Project deadline moved to Tuesday")
        print(f"   Memories before consolidation: {len(memory_store.memories)}")
        consolidated_count = memory_store.consolidate_memories()
        print(f"   Memories after consolidation: {consolidated_count}")
    else:
        print("   No memories to consolidate")
else:
    print("   Note: consolidate_memories method not available (run enhancement cell first)")

# Test 4: Context compaction
print("\n4. Testing context compaction:")
if hasattr(memory_store, 'compact_context'):
    compacted = memory_store.compact_context(max_memories=5)
    print(f"   Compacted context: {len(compacted)} most relevant memories")
else:
    print("   Note: compact_context method not available (run enhancement cell first)")

# Test 5: Preference learning from patterns
print("\n5. Testing preference learning from patterns:")
if hasattr(memory_store, 'learn_from_patterns'):
    classification_history = [
        {"app": "Banking App", "classification": "URGENT"},
        {"app": "Banking App", "classification": "URGENT"},
        {"app": "Banking App", "classification": "URGENT"},
        {"app": "Social Media", "classification": "IRRELEVANT"},
        {"app": "Social Media", "classification": "IRRELEVANT"},
        {"app": "Social Media", "classification": "IRRELEVANT"},
    ]
    memory_store.learn_from_patterns(classification_history)
    if hasattr(memory_store, 'get_user_preferences'):
        print(f"   Learned preferences: {memory_store.get_user_preferences()}")
    else:
        print("   Pattern learning completed")
else:
    print("   Note: learn_from_patterns method not available (run enhancement cell first)")

print("\n" + "="*70)
print("✅ Enhanced features demonstration complete")
print("="*70 + "\n")



🔧 Testing Enhanced Features

1. Testing retrieve_user_preferences() tool:
   Current preferences: {'always_urgent_apps': [], 'always_block_apps': [], 'preferred_categories': [], 'blocked_keywords': []}

2. Testing preference learning:
✅ Added preference: always_urgent_apps = Banking App
✅ Added preference: always_block_apps = Social Media
   Updated preferences: {'always_urgent_apps': ['Banking App'], 'always_block_apps': ['Social Media'], 'preferred_categories': [], 'blocked_keywords': []}

3. Testing memory consolidation:
💾 Stored memory: Project deadline moved to Tuesday
   Memories before consolidation: 2
   Memories after consolidation: 2

4. Testing context compaction:
   Compacted context: 2 most relevant memories

5. Testing preference learning from patterns:
   Learned preferences: {'always_urgent_apps': ['Banking App'], 'always_block_apps': ['Social Media'], 'preferred_categories': [], 'blocked_keywords': []}

✅ Enhanced features demonstration complete



In [18]:
# ============================================================================
# STEP 6: Agent Evaluation Framework (LLM-as-Judge)
# ============================================================================
# This implements automated evaluation of agent performance using:
# - Golden test suite with labeled notifications
# - LLM-as-judge evaluation framework
# - Comprehensive metrics (classification accuracy, action correctness, memory quality)
# - Automated scoring and reporting

from typing import Dict, List, Optional, Tuple
import time
from dataclasses import dataclass
import json

@dataclass
class TestCase:
    """Represents a test case with expected results"""
    id: str
    notification_text: str
    app: str
    title: str
    body: str
    expected_classification: str  # URGENT, IRRELEVANT, or LESS_URGENT
    expected_action: str  # display_urgent, block, or save_memory
    expected_extracted_fact: Optional[str] = None  # For LESS_URGENT cases
    reasoning: str = ""  # Why this classification is expected

# Golden test suite - labeled notifications for evaluation
GOLDEN_TEST_SUITE = [
    TestCase(
        id="test_001",
        notification_text="I received a notification: App: Banking App, Title: Security Alert, Body: Your bank flagged suspicious activity on your account. Please verify immediately.",
        app="Banking App",
        title="Security Alert",
        body="Your bank flagged suspicious activity on your account. Please verify immediately.",
        expected_classification="URGENT",
        expected_action="display_urgent",
        reasoning="Security alerts from banking apps require immediate attention"
    ),
    TestCase(
        id="test_002",
        notification_text="I received a notification: App: Social Media, Title: New Like, Body: 3 new people liked your photo.",
        app="Social Media",
        title="New Like",
        body="3 new people liked your photo.",
        expected_classification="IRRELEVANT",
        expected_action="block",
        reasoning="Social media likes are noise and don't require attention"
    ),
    TestCase(
        id="test_003",
        notification_text="I received a notification: App: Project Manager, Title: Deadline Update, Body: Your project deadline has moved to Tuesday.",
        app="Project Manager",
        title="Deadline Update",
        body="Your project deadline has moved to Tuesday.",
        expected_classification="LESS_URGENT",
        expected_action="save_memory",
        expected_extracted_fact="Project deadline moved to Tuesday",
        reasoning="Project updates are important but not immediately urgent"
    ),
    TestCase(
        id="test_004",
        notification_text="I received a notification: App: Email, Title: Meeting Reminder, Body: You have a meeting in 15 minutes with the CEO.",
        app="Email",
        title="Meeting Reminder",
        body="You have a meeting in 15 minutes with the CEO.",
        expected_classification="URGENT",
        expected_action="display_urgent",
        reasoning="Time-sensitive meeting reminders require immediate attention"
    ),
    TestCase(
        id="test_005",
        notification_text="I received a notification: App: Shopping App, Title: Flash Sale, Body: 50% off all items! Limited time offer!",
        app="Shopping App",
        title="Flash Sale",
        body="50% off all items! Limited time offer!",
        expected_classification="IRRELEVANT",
        expected_action="block",
        reasoning="Promotional content is typically noise"
    ),
    TestCase(
        id="test_006",
        notification_text="I received a notification: App: Calendar, Title: Event Tomorrow, Body: Team standup meeting scheduled for tomorrow at 10 AM.",
        app="Calendar",
        title="Event Tomorrow",
        body="Team standup meeting scheduled for tomorrow at 10 AM.",
        expected_classification="LESS_URGENT",
        expected_action="save_memory",
        expected_extracted_fact="Team standup meeting tomorrow at 10 AM",
        reasoning="Future events are important to remember but not urgent"
    ),
    TestCase(
        id="test_007",
        notification_text="I received a notification: App: Health App, Title: Medication Reminder, Body: Time to take your daily medication.",
        app="Health App",
        title="Medication Reminder",
        body="Time to take your daily medication.",
        expected_classification="URGENT",
        expected_action="display_urgent",
        reasoning="Health-related reminders are time-sensitive and important"
    ),
    TestCase(
        id="test_008",
        notification_text="I received a notification: App: News App, Title: Breaking News, Body: Local weather update: Sunny skies expected today.",
        app="News App",
        title="Breaking News",
        body="Local weather update: Sunny skies expected today.",
        expected_classification="IRRELEVANT",
        expected_action="block",
        reasoning="Generic news updates are typically not urgent"
    ),
]

class EvaluationJudge:
    """LLM-as-judge for evaluating agent performance"""
    
    def __init__(self):
        self.judge_agent = LlmAgent(
            name="evaluation_judge",
            model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
            instruction="""You are an Evaluation Judge for the Focus Filter notification system.

Your job is to evaluate whether an agent's classification and actions match the expected behavior.

You will receive:
1. The test case (notification and expected results)
2. The agent's actual output (classification, action, extracted fact)

Evaluate:
1. **Classification Match**: Does the agent's classification match the expected classification?
   - URGENT, IRRELEVANT, or LESS_URGENT
   - Consider if the classification is reasonable even if not exact match

2. **Action Match**: Does the agent's action match the expected action?
   - display_urgent, block, or save_memory
   - Action should align with classification

3. **Memory Extraction Quality** (for LESS_URGENT cases):
   - Is the extracted fact accurate and useful?
   - Does it capture the key information?

Respond in JSON format:
{
    "classification_match": true/false,
    "classification_score": 0.0-1.0,
    "action_match": true/false,
    "action_score": 0.0-1.0,
    "memory_quality": 0.0-1.0 (only for LESS_URGENT),
    "overall_score": 0.0-1.0,
    "reasoning": "explanation of scores"
}""",
            tools=[]
        )
        self.judge_runner = Runner(
            app_name="FocusFilter_Evaluation",
            agent=self.judge_agent,
            session_service=InMemorySessionService()
        )
    
    async def evaluate_single_case(
        self, 
        test_case: TestCase, 
        actual_classification: str,
        actual_action: str,
        actual_extracted_fact: Optional[str] = None
    ) -> Dict:
        """Evaluate a single test case using LLM-as-judge"""
        
        evaluation_prompt = f"""Evaluate this agent performance:

TEST CASE:
- Notification: {test_case.app} - {test_case.title}: {test_case.body}
- Expected Classification: {test_case.expected_classification}
- Expected Action: {test_case.expected_action}
{f"- Expected Extracted Fact: {test_case.expected_extracted_fact}" if test_case.expected_extracted_fact else ""}
- Reasoning: {test_case.reasoning}

AGENT OUTPUT:
- Actual Classification: {actual_classification}
- Actual Action: {actual_action}
{f"- Actual Extracted Fact: {actual_extracted_fact}" if actual_extracted_fact else ""}

Evaluate the agent's performance and provide scores in JSON format."""
        
        try:
            session = await self.judge_runner.session_service.create_session(
                app_name=self.judge_runner.app_name,
                user_id="evaluator",
                session_id=f"eval_{test_case.id}"
            )
            
            message = types.Content(
                role="user",
                parts=[types.Part(text=evaluation_prompt)]
            )
            
            judge_response = ""
            async for event in self.judge_runner.run_async(
                user_id="evaluator",
                session_id=session.id,
                new_message=message
            ):
                if event.content and event.content.parts:
                    if event.content.parts[0].text:
                        judge_response += event.content.parts[0].text
            
            # Try to parse JSON from response
            try:
                # Extract JSON from response (might be wrapped in markdown)
                if "```json" in judge_response:
                    json_start = judge_response.find("```json") + 7
                    json_end = judge_response.find("```", json_start)
                    judge_response = judge_response[json_start:json_end].strip()
                elif "```" in judge_response:
                    json_start = judge_response.find("```") + 3
                    json_end = judge_response.find("```", json_start)
                    judge_response = judge_response[json_start:json_end].strip()
                
                evaluation_result = json.loads(judge_response)
            except:
                # Fallback: simple scoring based on exact matches
                evaluation_result = {
                    "classification_match": actual_classification.upper() == test_case.expected_classification.upper(),
                    "classification_score": 1.0 if actual_classification.upper() == test_case.expected_classification.upper() else 0.0,
                    "action_match": actual_action == test_case.expected_action,
                    "action_score": 1.0 if actual_action == test_case.expected_action else 0.0,
                    "memory_quality": 0.5,  # Default
                    "overall_score": 0.5,
                    "reasoning": "Fallback scoring used (JSON parsing failed)"
                }
            
            return {
                "test_case_id": test_case.id,
                "evaluation": evaluation_result,
                "expected": {
                    "classification": test_case.expected_classification,
                    "action": test_case.expected_action,
                    "extracted_fact": test_case.expected_extracted_fact
                },
                "actual": {
                    "classification": actual_classification,
                    "action": actual_action,
                    "extracted_fact": actual_extracted_fact
                }
            }
            
        except Exception as e:
            return {
                "test_case_id": test_case.id,
                "error": str(e),
                "evaluation": {
                    "classification_match": False,
                    "classification_score": 0.0,
                    "action_match": False,
                    "action_score": 0.0,
                    "overall_score": 0.0,
                    "reasoning": f"Evaluation error: {e}"
                }
            }

class EvaluationFramework:
    """Comprehensive evaluation framework for agent performance"""
    
    def __init__(self):
        self.judge = EvaluationJudge()
        self.results: List[Dict] = []
    
    async def run_evaluation(self, test_suite: List[TestCase] = None) -> Dict:
        """Run evaluation on the test suite"""
        if test_suite is None:
            test_suite = GOLDEN_TEST_SUITE
        
        print(f"\n{'='*70}")
        print(f"🔬 AGENT EVALUATION FRAMEWORK")
        print(f"{'='*70}")
        print(f"\n📋 Running evaluation on {len(test_suite)} test cases...\n")
        
        self.results = []
        
        for i, test_case in enumerate(test_suite, 1):
            print(f"[{i}/{len(test_suite)}] Processing: {test_case.id} - {test_case.app}")
            
            # Process notification through the agent system
            try:
                # Extract notification details
                notification_id = f"eval_{test_case.id}"
                
                # Run through the multi-agent system
                # We'll capture the results from the orchestration
                # For now, we'll simulate by calling the process function
                # and capturing the results
                
                # Start trace
                obs_manager.start_trace(notification_id, test_case.notification_text)
                
                # Run classification
                classification_start = time.time()
                session = await classification_runner.session_service.create_session(
                    app_name=classification_runner.app_name,
                    user_id="evaluator",
                    session_id=f"eval_{test_case.id}_classify"
                )
                
                message = types.Content(
                    role="user",
                    parts=[types.Part(text=test_case.notification_text)]
                )
                
                classification_result_text = ""
                async for event in classification_runner.run_async(
                    user_id="evaluator",
                    session_id=session.id,
                    new_message=message
                ):
                    if event.content and event.content.parts:
                        if event.content.parts[0].text:
                            classification_result_text += event.content.parts[0].text
                
                classification_data = parse_classification_result(classification_result_text)
                actual_classification = classification_data.get('classification', 'UNKNOWN')
                
                # Run action
                action_start = time.time()
                action_session = await action_runner.session_service.create_session(
                    app_name=action_runner.app_name,
                    user_id="evaluator",
                    session_id=f"eval_{test_case.id}_action"
                )
                
                action_message_text = f"""Notification Details:
- App: {test_case.app}
- Title: {test_case.title}
- Body: {test_case.body}

Classification Result:
- Classification: {actual_classification}
- Reasoning: {classification_data.get('reasoning', '')}
{f"- Key Fact: {classification_data.get('key_fact', '')}" if classification_data.get('key_fact') else ""}

Please execute the appropriate action based on the classification."""
                
                action_message = types.Content(
                    role="user",
                    parts=[types.Part(text=action_message_text)]
                )
                
                action_output = ""
                async for event in action_runner.run_async(
                    user_id="evaluator",
                    session_id=action_session.id,
                    new_message=action_message
                ):
                    if event.content and event.content.parts:
                        if event.content.parts[0].text:
                            action_output += event.content.parts[0].text
                
                # Determine actual action from output
                actual_action = "unknown"
                if "display_urgent_notification" in action_output.lower() or "urgent notification" in action_output.lower():
                    actual_action = "display_urgent"
                elif "block" in action_output.lower() or "blocked" in action_output.lower():
                    actual_action = "block"
                elif "save" in action_output.lower() or "memory" in action_output.lower() or "stored" in action_output.lower():
                    actual_action = "save_memory"
                
                actual_extracted_fact = classification_data.get('key_fact')
                
                # Evaluate using LLM-as-judge
                evaluation_result = await self.judge.evaluate_single_case(
                    test_case=test_case,
                    actual_classification=actual_classification,
                    actual_action=actual_action,
                    actual_extracted_fact=actual_extracted_fact
                )
                
                self.results.append(evaluation_result)
                
                # End trace
                obs_manager.end_trace()
                
                print(f"   ✅ Completed - Classification: {actual_classification}, Action: {actual_action}")
                
            except Exception as e:
                print(f"   ❌ Error: {e}")
                self.results.append({
                    "test_case_id": test_case.id,
                    "error": str(e),
                    "evaluation": {
                        "overall_score": 0.0,
                        "reasoning": f"Processing error: {e}"
                    }
                })
        
        # Calculate metrics
        metrics = self.calculate_metrics()
        
        # Print report
        self.print_evaluation_report(metrics)
        
        return {
            "results": self.results,
            "metrics": metrics
        }
    
    def calculate_metrics(self) -> Dict:
        """Calculate evaluation metrics from results"""
        if not self.results:
            return {}
        
        total = len(self.results)
        classification_correct = sum(
            1 for r in self.results 
            if r.get("evaluation", {}).get("classification_match", False)
        )
        action_correct = sum(
            1 for r in self.results 
            if r.get("evaluation", {}).get("action_match", False)
        )
        
        classification_scores = [
            r.get("evaluation", {}).get("classification_score", 0.0)
            for r in self.results
        ]
        action_scores = [
            r.get("evaluation", {}).get("action_score", 0.0)
            for r in self.results
        ]
        overall_scores = [
            r.get("evaluation", {}).get("overall_score", 0.0)
            for r in self.results
        ]
        
        # Memory quality (only for LESS_URGENT cases)
        memory_scores = [
            r.get("evaluation", {}).get("memory_quality", 0.0)
            for r in self.results
            if r.get("expected", {}).get("classification") == "LESS_URGENT"
        ]
        
        return {
            "total_tests": total,
            "classification_accuracy": classification_correct / total if total > 0 else 0.0,
            "action_accuracy": action_correct / total if total > 0 else 0.0,
            "avg_classification_score": sum(classification_scores) / len(classification_scores) if classification_scores else 0.0,
            "avg_action_score": sum(action_scores) / len(action_scores) if action_scores else 0.0,
            "avg_overall_score": sum(overall_scores) / len(overall_scores) if overall_scores else 0.0,
            "avg_memory_quality": sum(memory_scores) / len(memory_scores) if memory_scores else 0.0,
            "classification_correct": classification_correct,
            "action_correct": action_correct
        }
    
    def print_evaluation_report(self, metrics: Dict):
        """Print a formatted evaluation report"""
        print(f"\n{'='*70}")
        print(f"📊 EVALUATION REPORT")
        print(f"{'='*70}")
        print(f"\n📈 Overall Metrics:")
        print(f"   Total Tests: {metrics.get('total_tests', 0)}")
        print(f"   Overall Score: {metrics.get('avg_overall_score', 0.0):.2%}")
        print(f"\n🏷️  Classification Metrics:")
        print(f"   Accuracy: {metrics.get('classification_accuracy', 0.0):.2%} ({metrics.get('classification_correct', 0)}/{metrics.get('total_tests', 0)})")
        print(f"   Average Score: {metrics.get('avg_classification_score', 0.0):.2%}")
        print(f"\n⚡ Action Metrics:")
        print(f"   Accuracy: {metrics.get('action_accuracy', 0.0):.2%} ({metrics.get('action_correct', 0)}/{metrics.get('total_tests', 0)})")
        print(f"   Average Score: {metrics.get('avg_action_score', 0.0):.2%}")
        if metrics.get('avg_memory_quality', 0) > 0:
            print(f"\n💾 Memory Extraction Quality:")
            print(f"   Average Score: {metrics.get('avg_memory_quality', 0.0):.2%}")
        print(f"\n{'='*70}\n")
        
        # Print individual results
        print(f"\n📋 Individual Test Results:\n")
        for result in self.results:
            test_id = result.get("test_case_id", "unknown")
            eval_data = result.get("evaluation", {})
            expected = result.get("expected", {})
            actual = result.get("actual", {})
            
            status = "✅" if eval_data.get("overall_score", 0) >= 0.7 else "⚠️" if eval_data.get("overall_score", 0) >= 0.4 else "❌"
            
            print(f"{status} {test_id}:")
            print(f"   Expected: {expected.get('classification')} → {expected.get('action')}")
            print(f"   Actual: {actual.get('classification')} → {actual.get('action')}")
            print(f"   Score: {eval_data.get('overall_score', 0.0):.2%}")
            if eval_data.get("reasoning"):
                print(f"   Reasoning: {eval_data.get('reasoning')[:100]}...")
            print()

# Initialize evaluation framework
evaluation_framework = EvaluationFramework()

print("✅ Agent Evaluation Framework initialized")
print("   • Golden test suite ready ({len(GOLDEN_TEST_SUITE)} test cases)")
print("   • LLM-as-judge evaluation ready")
print("   • Metrics calculation ready")
print("   • Run evaluation_framework.run_evaluation() to start")

✅ Agent Evaluation Framework initialized
   • Golden test suite ready ({len(GOLDEN_TEST_SUITE)} test cases)
   • LLM-as-judge evaluation ready
   • Metrics calculation ready
   • Run evaluation_framework.run_evaluation() to start


In [19]:
# ============================================================================
# Run Agent Evaluation
# ============================================================================

# Run the full evaluation suite
evaluation_results = await evaluation_framework.run_evaluation()

# The evaluation framework will:
# 1. Process each test case through the multi-agent system
# 2. Evaluate results using LLM-as-judge
# 3. Calculate comprehensive metrics
# 4. Print a detailed report

print("\n✅ Evaluation complete!")
print("\nAccess results via: evaluation_results['results'] and evaluation_results['metrics']")

2025-12-02 02:43:30,488 - FocusFilter - INFO - 🔍 Trace started: trace_634db925
2025-12-02 02:43:30,492 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False



🔬 AGENT EVALUATION FRAMEWORK

📋 Running evaluation on 8 test cases...

[1/8] Processing: test_001 - Banking App


2025-12-02 02:43:31,000 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:31,012 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:43:31,800 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:31,803 - google_genai.types - WARNING - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-12-02 02:43:31,818 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False



🚨 URGENT NOTIFICATION
App: Banking App
Title: Security Alert
Body: Your bank flagged suspicious activity on your account. Please verify immediately.



2025-12-02 02:43:32,338 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:32,634 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:43:32,636 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-12-02 02:43:34,012 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:43:34,019 - FocusFilter - INFO - ✅ Trace completed: trace_634db925 in 3530.19ms
2025-12-02 02:43:34,023 - FocusFilter - INFO - 🔍 Trace started: trace_9e3e3186
2025-12-02 02:43:34,034 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:43:34,136 - google_genai._api_client - INFO - Retrying google.genai._api_client.BaseApiClient._async_request_once in 1.9124071518184285 seconds 

...

   ✅ Completed - Classification: URGENT, Action: display_urgent
[8/8] Processing: test_008 - News App


2025-12-02 02:46:06,092 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:46:06,104 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:46:06,204 - google_genai._api_client - INFO - Retrying google.genai._api_client.BaseApiClient._async_request_once in 1.1572651760070651 seconds as it raised ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota. Please migrate to Gemini 2.0 Flash Preview (Image Generation) (models/gemini-2.0-flash-preview-image-generation) for higher quota limits. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/doc

💾 Stored memory: Sunny skies expected today.


2025-12-02 02:47:06,024 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:47:06,032 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.0-flash-exp, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-12-02 02:47:06,035 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-12-02 02:47:07,678 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-12-02 02:47:07,685 - FocusFilter - INFO - ✅ Trace completed: trace_c44f0e1e in 64300.17ms


   ✅ Completed - Classification: LESS_URGENT, Action: save_memory

📊 EVALUATION REPORT

📈 Overall Metrics:
   Total Tests: 8
   Overall Score: 91.25%

🏷️  Classification Metrics:
   Accuracy: 87.50% (7/8)
   Average Score: 90.00%

⚡ Action Metrics:
   Accuracy: 87.50% (7/8)
   Average Score: 90.00%

💾 Memory Extraction Quality:
   Average Score: 100.00%



📋 Individual Test Results:

✅ test_001:
   Expected: URGENT → display_urgent
   Actual: URGENT → display_urgent
   Score: 100.00%
   Reasoning: The agent correctly classified the notification as URGENT and took the appropriate action of display...

✅ test_002:
   Expected: IRRELEVANT → block
   Actual: IRRELEVANT → block
   Score: 100.00%
   Reasoning: The agent correctly classified the social media notification as irrelevant and chose to block it, wh...

✅ test_003:
   Expected: LESS_URGENT → save_memory
   Actual: LESS_URGENT → save_memory
   Score: 100.00%
   Reasoning: The agent correctly classified the notification as LESS_URGEN

## Architecture Overview

This implementation demonstrates:

1. **Multi-Agent System**: Sequential agents (Classification → Action → Memory)
   - **Classification Agent**: Analyzes and classifies notifications
   - **Action Agent**: Executes actions based on classification
   - **Memory Agent**: Handles memory extraction and consolidation
2. **Custom Tools**: Three tools for notification management (display, block, save)
3. **Memory Management**: Simple in-memory storage (can be extended to vector DB)
4. **Orchestration Layer**: Coordinates sequential agent flow
5. **Agentic Loop**: Get Mission → Think → Act → Observe pattern

## Multi-Agent Flow

```
Notification Input
    ↓
[Classification Agent] → Classifies as URGENT/IRRELEVANT/LESS_URGENT
    ↓
[Action Agent] → Executes appropriate action (display/block/save)
    ↓
[Memory Agent] → (Optional) Refines memory extraction for LESS_URGENT items
    ↓
Result
```

## Next Steps for Full Implementation

- ✅ Multi-agent architecture (COMPLETE)
- Add vector database for semantic memory search
- Add context engineering with few-shot examples
- Implement full observability with tracing
- Add agent evaluation framework with LLM-as-judge
- Add user preference learning
